* Raw sensor data was recorded with 2 devices:
1. Chest-worn device(RespiBAN) 
2. Wrist-worn device(Empatica E4)
* The RespiBAN device provides the following sensor data: electrocardiogram (ECG), electrodermal activity (EDA), electromyogram (EMG), respiration, body temperature, and three-axis acceleration. All signals are sampled at 700 Hz.


* Empatica E4 device provides the following sensor data: blood volume pulse (BVP, 64 Hz), electrodermal activity (EDA, 4 Hz), body temperature (4 Hz), and three-axis acceleration (32 Hz).

## Steps for Data Cleaning and get structured data

In [1]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import pathlib
import shutil

2022-07-15 01:08:41.214435: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-15 01:08:41.214526: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [18]:
with open('/home/harinadh_asr2/project/WESAD/S7/S7.pkl', 'rb') as f:
    data = pickle.load(f, encoding='latin1')

In [12]:
data['signal']['chest']['EMG'].shape[0]/2100
# 'ACC' : 3, 'ECG' 1: , 'EDA' : 1, 'EMG': 1, 'RESP': 1, 'Temp': 1

1746.0

In [19]:
data['signal']['chest']

{'ACC': array([[ 1.06960011, -0.014     ,  1.4612    ],
        [ 1.06539989, -0.01020002,  1.42219996],
        [ 1.06539989,  0.00419998,  1.32019997],
        ...,
        [ 0.84660006, -0.10820001, -0.05739999],
        [ 0.84220004, -0.11379999, -0.08139998],
        [ 0.83340001, -0.11519998, -0.10259998]]),
 'ECG': array([[-0.01922607],
        [-0.01693726],
        [-0.01615906],
        ...,
        [ 0.08638   ],
        [ 0.13993835],
        [ 0.23899841]]),
 'EMG': array([[-0.04884338],
        [-0.00700378],
        [ 0.07177734],
        ...,
        [ 0.19976807],
        [ 0.22764587],
        [ 0.21807861]]),
 'EDA': array([[12.50686646],
        [12.46871948],
        [12.50686646],
        ...,
        [13.09013367],
        [13.08135986],
        [13.08021545]]),
 'Temp': array([[33.99353 ],
        [33.93268 ],
        [33.970703],
        ...,
        [34.9433  ],
        [34.940247],
        [34.947906]], dtype=float32),
 'Resp': array([[1.83258057],
        [1

In [ ]:
# data['signal']['wrist']['BVP'].shape
# np.sum()/700
2198*700
# data = data['label'][data['label']==3]
for i in range(0,len(data),700):
    print(np.mean(data[i:i+700]))

In [24]:
import math

math.ceil(len(data)/700)

373

In [8]:
dataset_path = '/home/harinadh_asr2/project/WESAD'

In [ ]:
pickle_files = []
for root, dirnames, filenames in os.walk(dataset_path, topdown=True):
    # print(dirnames)
    for file_name in filenames :
        if file_name.endswith('.pkl'):
            print(file_name)
            pickle_files.append(os.path.join(root,file_name))

In [15]:
# for file in pickle_files:

In [16]:
class read_subject_data:
    def __init__(self, file_path, subject_no):
        """ Read Subject data from pkl file"""
        self.subject_no=subject_no
        with open(file_path, 'rb') as file:
            data=pickle.load(file, encoding='latin1')
        self.data= data
        self.keys=self.data.keys() #['label', 'subject', 'signal']
        self.signal_keys=self.data['signal'].keys() #['wrist', 'chest']
        self.chest_sensor_keys = self.data['signal']['chest'].keys() #['ACC', 'ECG', 'EDA', 'EMG', 'Resp', 'Temp']
        self.wrist_sensor_keys = self.data['signal']['wrist'].keys() # ['ACC', 'BVP', 'EDA', 'TEMP']
        self.labels = self.data['label']
    
    def get_wrist_data(self):
        wrist_data = self.data.get('signal',{}).get('wrist',[])
        return wrist_data
    
    def get_chest_data(self):
        chest_data = self.data.get('signal').get('chest', [])
        return chest_data
    
    def extract_chest_features(self, ):
        pass

In [17]:
data['signal']['chest'].keys()

dict_keys(['ACC', 'ECG', 'EMG', 'EDA', 'Temp', 'Resp'])

In [28]:
data['signal']['chest']['ACC'].shape

(4255300, 3)

In [27]:
data['signal']['chest']['ECG'].shape

(4255300, 1)

In [26]:
data['signal']['chest']['EMG'].shape

(4255300, 1)

In [25]:
data['signal']['chest']['EDA'].shape

(4255300, 1)

In [24]:
data['signal']['chest']['Resp'].shape

(4255300, 1)

In [29]:
data['signal']['chest']['Temp'].shape

(4255300, 1)

In [1]:
# np.sum(data['label'][0:700])
4255300/700

6079.0

In [45]:
np.sum(data['label'][510*700:511*700])

700

In [27]:
chest_data_dict = data['signal']['chest']

In [28]:
chest_data = np.concatenate([chest_data_dict[key] for key in chest_data_dict], axis=1)

In [29]:
chest_data.shape

(4144000, 8)

In [31]:
chest_data
filter_criteria = np.where(data['label']<5)
filter_criteria[0].shape[0]/700

5746.001428571429

In [34]:
data['label'].shape

(4144000,)

In [68]:

np.where(data['label']==0)[0].shape[0]//700

2739

In [60]:
np.where(data['label']==1)

(array([ 214583,  214584,  214585, ..., 1015380, 1015381, 1015382]),)

In [62]:
a = np.array([1,2,3])
np.where(a==2)

(array([1]),)

In [58]:
np.asarray([1,2,3])

array([1, 2, 3])

In [74]:
chest_data_dict["ACC"][filter_criteria]

(12495000,)

In [75]:
chest_data_dict.keys()

dict_keys(['ACC', 'ECG', 'EMG', 'EDA', 'Temp', 'Resp'])

In [84]:
def aggreage_feature(feature, frequency=700):
    mean_features = np.zeros(len(feature)//frequency)
    std_features = np.zeros(len(feature)//frequency)
    max_features = np.zeros(len(feature)//frequency)
    min_features = np.zeros(len(feature)//frequency)

    idx =0
    for i in range(0, len(feature),frequency):
        mean_features[idx] = np.mean(feature[i:i+700])
        std_features[idx] = np.std(feature[i:i+700])
        max_features[idx] = np.amax(feature[i:i+700])
        min_features[idx] = np.amin(feature[i:i+700])
        idx+=1
    
    return np.column_stack([max_features, min_features, std_features, mean_features])

In [17]:
# chest_data[filter_criteria].shape
data

(260401,)

In [107]:
chest_data[filter_criteria][:,0].shape

(4165000,)

In [109]:
aggreage_feature(chest_data[filter_criteria][:,0]).shape

(5950, 4)

In [110]:
data = np.apply_along_axis(aggreage_feature,0,chest_data[filter_criteria])

In [120]:
# b = np.array([[1,2,3], [4,5,6]])
# def my_func(arr):
#     return np.mean(arr)

# np.apply_along_axis(my_func, 0, b)

data.reshape(data.shape[0],-1).shape

(5950, 32)

In [91]:
chest_data[filter_criteria].shape

(4165000, 8)

In [101]:
# 4165000/700
b.shape

(2, 3)

In [118]:
b

array([[1, 2, 3],
       [4, 5, 6]])

In [119]:
b.reshape(b.shape[0]*2,b.shape[1]//2)

ValueError: cannot reshape array of size 6 into shape (4,1)

In [4]:
DATASET_PATH = "./WESAD/"
subject_no = "S2"

In [122]:
with open(f'{DATASET_PATH}/{subject_no}_features.npy','wb') as f:
            np.save(f, chest_data)

In [5]:
pickle_files = {}
for root, dirnames, filenames in os.walk(DATASET_PATH, topdown=True):
    for file_name in filenames :
        if file_name.endswith('.pkl'):
            subject_no = file_name.split('.')[0]
            pickle_files[subject_no] = os.path.join(root,file_name)

In [6]:
pickle_files

{'S3': './WESAD/S3/S3.pkl',
 'S15': './WESAD/S15/S15.pkl',
 'S7': './WESAD/S7/S7.pkl',
 'S17': './WESAD/S17/S17.pkl',
 'S16': './WESAD/S16/S16.pkl',
 'S6': './WESAD/S6/S6.pkl',
 'S8': './WESAD/S8/S8.pkl',
 'S4': './WESAD/S4/S4.pkl',
 'S5': './WESAD/S5/S5.pkl',
 'S11': './WESAD/S11/S11.pkl',
 'S9': './WESAD/S9/S9.pkl',
 'S10': './WESAD/S10/S10.pkl',
 'S13': './WESAD/S13/S13.pkl',
 'S2': './WESAD/S2/S2.pkl',
 'S14': './WESAD/S14/S14.pkl'}

In [1]:
import tensorflow as tf

2022-06-19 19:23:29.271153: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-19 19:23:29.271237: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
var = tf.Variable([0.0, 0.0, 0.0])

2022-06-19 19:24:12.525260: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-06-19 19:24:12.525483: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-06-19 19:24:12.525558: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (instance-1): /proc/driver/nvidia/version does not exist
2022-06-19 19:24:12.527894: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
#loss function == mean square error --> differnetiate mean square error - w -alpha*de/dw -->[[w1,w2,w3],
#                                                                                            [w4,w5,w6]]
# Tensors can be strings, too here is a scalar string.
scalar_string_tensor = tf.constant("Gray wolf")
print(scalar_string_tensor)


tf.Tensor(b'Gray wolf', shape=(), dtype=string)


In [ ]:
# Sparse tensors store values by index in a memory-efficient manner
sparse_tensor = tf.sparse.SparseTensor(indices=[[0, 0], [1, 2]],
                                       values=[1, 2],
                                       dense_shape=[3, 4])
print(sparse_tensor, "\n")

# You can convert sparse tensors to dense
print(tf.sparse.to_dense(sparse_tensor))


<module 'tensorflow' from '/home/harinadh_asr2/miniconda3/envs/tff/lib/python3.9/site-packages/tensorflow/__init__.py'>

In [17]:
tf.constant([[12,2,45],[13,34,45]]).numpy()

array([[12,  2, 45],
       [13, 34, 45]], dtype=int32)

In [2]:
t0 = 33
t1= tf.Variable(44)


2022-07-15 01:17:14.200949: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-07-15 01:17:14.201109: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-07-15 01:17:14.201149: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (instance-1): /proc/driver/nvidia/version does not exist
2022-07-15 01:17:14.202214: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
t1.numpy()

44

In [ ]:
EDA, ECG, EMG, RESP, TEMP, ACC3

In [9]:
file_path = '../WESAD/S10_train_data.npy'

In [11]:
data = np.load(file_path, allow_pickle=True)

In [16]:
chest_features = data.item()['features'][:,0:28]

In [17]:
pd.DataFrame(chest_features).describe()

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
count,1.598000e+03,1.598000e+03,1.598000e+03,1.598000e+03,1.598000e+03,1.598000e+03,1.598000e+03,1.598000e+03,1.598000e+03,1.598000e+03,...,1.598000e+03,1.598000e+03,1.598000e+03,1.598000e+03,1.598000e+03,1.598000e+03,1.598000e+03,1.598000e+03,1.598000e+03,1.598000e+03
mean,-6.225030e-17,7.781288e-18,-8.781739e-17,2.223225e-17,9.337545e-17,-1.333935e-17,8.648346e-16,2.000903e-17,-8.225933e-17,2.667870e-17,...,-1.578490e-16,-1.213881e-15,3.283703e-15,8.234826e-15,1.517573e-14,1.133845e-15,-3.907318e-15,-6.530724e-16,-2.890193e-17,1.556258e-16
std,1.000313e+00,1.000313e+00,1.000313e+00,1.000313e+00,1.000313e+00,1.000313e+00,1.000313e+00,1.000313e+00,1.000313e+00,1.000313e+00,...,1.000313e+00,1.000313e+00,1.000313e+00,1.000313e+00,1.000313e+00,1.000313e+00,1.000313e+00,1.000313e+00,1.000313e+00,1.000313e+00
min,-2.521452e+00,-2.645645e+00,-1.432691e+00,-3.056233e+00,-1.449710e+00,-8.583526e-01,-1.454386e+00,-3.251367e+00,-2.884848e+00,-3.605213e+00,...,-6.449375e-01,-2.092762e+00,-1.402364e+00,-2.271589e+00,-1.984441e+00,-1.264856e+00,-2.406195e+00,-3.067837e+00,-1.374123e+00,-4.870385e+00
25%,-8.582162e-01,-3.400132e-01,-8.402137e-01,-6.850510e-01,-5.473840e-01,-7.394194e-01,-6.324868e-01,-7.377497e-01,-9.520606e-01,-2.298624e-01,...,-3.748433e-01,-6.823917e-01,-5.226256e-01,-5.324612e-01,-6.968661e-01,-7.266844e-01,-8.911709e-01,-2.746852e-01,-8.590827e-01,-6.334492e-01
50%,3.900127e-01,9.343119e-02,-1.575012e-01,-9.381462e-02,-2.143825e-01,-6.099920e-01,-5.368088e-01,3.996388e-02,5.454043e-01,7.698687e-02,...,-2.350623e-01,-2.673495e-02,-1.848186e-01,-1.172259e-02,-4.144511e-01,-1.523594e-01,5.034635e-01,9.784251e-02,-1.147752e-01,-7.257837e-03
75%,9.643404e-01,6.980658e-01,1.039187e+00,5.260177e-01,1.938128e-01,1.022425e+00,1.058338e+00,5.519488e-01,1.020572e+00,7.321032e-01,...,5.507857e-02,7.356661e-01,1.537389e-01,4.410339e-01,9.542629e-01,4.426202e-01,1.025522e+00,7.359198e-01,1.045620e+00,6.689597e-01
max,3.108750e+00,3.225883e+00,2.425931e+00,3.825655e+00,1.018923e+01,2.040353e+00,1.801804e+00,4.644978e+00,1.285215e+00,1.936913e+00,...,1.285085e+01,5.013780e+00,1.039615e+01,1.369851e+01,2.363858e+00,9.960628e+00,1.288818e+00,1.989393e+00,2.070848e+00,4.196384e+00
